
# Выбросы и новизна

Строго говоря, в анализе данных есть два направления, которые занимаются поиском аномалий:
- поиск выбросов (Outlier Detection)
- поиск «новизны» (Novelty Detection).

Как и выброс, «новый объект» — это объект, который отличается по своим свойствам от объектов (обучающей) выборки.
Но  в отличие от выброса, его в самой выборке пока нет (он появится через некоторое время, и задача как раз и заключается в том,
чтобы обнаружить его при появлении). Например, если вы анализируете замеры температуры и отбрасываете аномально большие или маленькие,
то Вы боретесь с выбросами. А если Вы создаёте алгоритм, который для каждого нового замера оценивает, насколько он похож на прошлые, и выбрасывает
аномальные — Вы «боретесь с новизной».

*Выбросы* являются следствием:

- ошибок в данных (неточности измерения, округления, неверной записи и т.п.)
- наличия шумовых объектов (неверно классифицированных объектов)
- присутствия объектов «других» выборок (например, показаниями сломавшегося датчика).

Новизна, как правило, появляется в результате принципиально нового поведения объекта. Скажем, если наши объекты – описания работы системы, то после проникновения в неё вируса объекты становятся «новизной». Ещё пример – описания работы двигателя после поломки. Здесь важно понимать, что «новизна» называется новизной по той причине, что такие описания для нас абсолютно новые, а новые они потому, что мы не можем в обучающей выборке иметь информацию о всевозможных заражениях вирусами или всевозможных поломках. Формирование такой обучающей выборки трудозатратно и часто не имеет смысла. Зато можно набрать достаточно большую выборку примеров нормальной (штатной) работы системы или механизма.

Приложений здесь море:

- Обнаружение подозрительных банковских операций (Credit-card Fraud)
- Обнаружение вторжений (Intrusion Detection)
- Обнаружение нестандартных игроков на бирже (инсайдеров)
- Обнаружение неполадок в механизмах по показаниям датчиков
- Медицинская диагностика (Medical Diagnosis)
- Сейсмология